In [56]:
import pandas as pd
import numpy as np


In [57]:
df = pd.read_csv('./diabetes.csv')

In [58]:
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [59]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [60]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [167]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Standard scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Print the scaled features
print(X_scaled)


[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
import random
import keras_tuner as kt

# Fix the seed values
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [94]:
# we can automate this 
# Create a 1-layer neural network
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [95]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [96]:

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1,validation_data=(X_test,y_test))


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3765 - loss: 0.7509 - val_accuracy: 0.5519 - val_loss: 0.6891
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5989 - loss: 0.6726 - val_accuracy: 0.6623 - val_loss: 0.6370
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7163 - loss: 0.6202 - val_accuracy: 0.7208 - val_loss: 0.6008
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7406 - loss: 0.5828 - val_accuracy: 0.7208 - val_loss: 0.5752
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7469 - loss: 0.5555 - val_accuracy: 0.7273 - val_loss: 0.5569
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7419 - loss: 0.5364 - val_accuracy: 0.7403 - val_loss: 0.5438
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7492 - loss: 0.5193 - val_accuracy: 0.7468 - val_loss: 0.5344
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7500 - loss: 0.5073 - val_accuracy: 0.7468 - val_loss:

In [97]:

# Predict the outcomes on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [98]:

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7597402597402597


1) selete optimizer
2) no of node in a layer
3) how to select no of layer
4) all in one model

In [99]:
# !pip install keras-tuner

In [100]:
import keras_tuner as kt

In [116]:
# Define model builder function
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    # model.add(Dense(50, activation='relu'))
    # model.add(Dense(25, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Tune the optimizer choice
    optimizer = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [117]:
# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=15,  # Number of trials (experiments)
)

In [118]:
tuner.search(X_train, y_train,
             validation_data=(X_test, y_test),
             epochs=10,
             verbose=2)  # Change verbose to 2


Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 06s


In [130]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=20)[0]
print("Best optimizer:", best_hps.get("optimizer"))

Best optimizer: rmsprop


In [131]:
# now getting best model 
model = tuner.get_best_models(num_models=1)[0]

In [132]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [133]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1,validation_data=(X_test,y_test))


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7646 - loss: 0.4788 - val_accuracy: 0.7987 - val_loss: 0.5040
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7617 - loss: 0.4711 - val_accuracy: 0.7922 - val_loss: 0.5026
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7567 - loss: 0.4678 - val_accuracy: 0.7857 - val_loss: 0.5022
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7567 - loss: 0.4652 - val_accuracy: 0.7857 - val_loss: 0.5024
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7587 - loss: 0.4609 - val_accuracy: 0.7857 - val_loss: 0.5027
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7583 - loss: 0.4614 - val_accuracy: 0.7922 - val_loss: 0.5034
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7578 - loss: 0.4600 - val_accuracy: 0.7857 - val_loss: 0.5043
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7589 - loss: 0.4587 - val_accuracy: 0.7792 - val_loss:

now for number of neuron

In [144]:
def build_model(hp):
    model = Sequential()
    # units = hp.Int(initital,end,steps)
    units = hp.Int('units',8,132,step=8)
    model.add(Dense(units=units, activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [145]:
# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,  # Number of trials (experiments)
)

In [146]:
tuner.search(X_train, y_train,
             validation_data=(X_test, y_test),
             epochs=10,
             verbose=2)  # Change verbose to 2


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 12s


In [149]:
best_hps = tuner.get_best_hyperparameters(num_trials=20)[0]
print("Best units:", best_hps.get("units"))

Best units: 72


In [150]:
# now for number of layer

In [151]:
def build_model(hp):
    model = Sequential()
    
    # Tune number of hidden layers (between 1 and 5)
    num_layers = hp.Int('num_layers', 1, 5)
    
    for i in range(num_layers):
        # Tune units per layer (8 to 128, step of 8)
        units = hp.Int(f'units_{i}', 8, 128, step=8)
        model.add(Dense(units=units, activation='relu'))
    
    # Output layer (1 neuron, sigmoid activation for binary classification)
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [155]:
# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,  # Number of trials (experiments)
)

In [156]:
tuner.search(X_train, y_train,
             validation_data=(X_test, y_test),
             epochs=10,
             verbose=2)  # Change verbose to 2


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 13s


In [157]:
# now for all hyperparameter

In [176]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    
    # Tune number of hidden layers (1 to 5)
    num_layers = hp.Int('num_layers', 1, 5)

    for i in range(num_layers):
        # Tune units per layer (8 to 128, step=8)
        units = hp.Int(f'units_{i}', 8, 128, step=8)
        
        # Tune activation function for each layer
        activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid', 'leaky_relu'])
        
        # Add Dense layer
        if activation == 'leaky_relu':
            model.add(Dense(units=units))
            model.add(LeakyReLU(alpha=0.01))  # Add LeakyReLU separately
        else:
            model.add(Dense(units=units, activation=activation))
        
        # Tune dropout rate (0.1 to 0.5 in steps of 0.1)
        dropout_rate = hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)
        model.add(Dropout(rate=dropout_rate))

    # Output layer (1 neuron, sigmoid for binary classification)
    model.add(Dense(1, activation='sigmoid'))

    # Tune optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    
    # Tune learning rate (log scale: 0.0001 to 0.01)
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')

    # Create optimizer with tuned learning rate
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.SGD(learning_rate=learning_rate)
    
    # Compile model
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [177]:
# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,  # Number of trials (experiments)
)

In [178]:
tuner.search(X_train, y_train,
             validation_data=(X_test, y_test),
             epochs=10,
             verbose=2)  # Change verbose to 2


Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 14s


In [179]:
# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best number of layers:", best_hps.get("num_layers"))
print("Best optimizer:", best_hps.get("optimizer"))
print("Best learning rate:", best_hps.get("learning_rate"))

Best number of layers: 1
Best optimizer: adam
Best learning rate: 0.00030658625710638904


In [180]:
model = tuner.get_best_models(num_models=1)[0]

/home/sharyar/Desktop/Aritificial_Intelligence/myenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [182]:
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1,validation_data=(X_test,y_test))


Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7589 - loss: 0.4780 - val_accuracy: 0.7468 - val_loss: 0.4920
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7579 - loss: 0.4838 - val_accuracy: 0.7532 - val_loss: 0.4924
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7611 - loss: 0.4834 - val_accuracy: 0.7597 - val_loss: 0.4929
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7633 - loss: 0.4772 - val_accuracy: 0.7597 - val_loss: 0.4934
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7676 - loss: 0.4831 - val_accuracy: 0.7597 - val_loss: 0.4940
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7685 - loss: 0.4768 - val_accuracy: 0.7597 - val_loss: 0.4942
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7699 - loss: 0.4761 - val_accuracy: 0.7597 - val_loss: 0.4947
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7549 - loss: 0.4772 - val_accuracy: 0.7597 - v